In [8]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import words

In [9]:
# Ensure necessary nltk resources are available
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package punkt to /Users/sambickel-
[nltk_data]     barlow/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sambickel-
[nltk_data]     barlow/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sambickel-
[nltk_data]     barlow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /Users/sambickel-
[nltk_data]     barlow/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
items_25 = pd.read_csv("/Users/sambickel-barlow/Desktop/PP422/Final Project/items_2025.csv")

In [6]:
pts = items_25['product_title'].tolist()
pts_df = pd.DataFrame(pts, columns=['title'])

In [10]:
stop_words = stopwords.words('english')
english_words = set(words.words())

In [11]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    tokens = [word for word in tokens if word in english_words]  # Remove non-english words
    return " ".join(tokens)

In [12]:
# Apply preprocessing
pts_df['processed'] = pts_df['title'].apply(preprocess_text)

In [13]:
# Convert text to a document-term matrix using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pts_df['processed'])

In [14]:
# Define gendered words
male_words = {'man','mans','men', 'mens', 'male', 'males', 'masculine', 'boy', 'boys', 'him', 'his', 'he', 'hes'}
female_words = {'woman','womans', 'women', 'womens' 'female','females', 'feminine', 'girl', 'girls', 'her', 'hers', 'she', 'shes'}

In [15]:
english_words.update(male_words)
english_words.update(female_words)

In [16]:
# Function to evaluate topic separation
def evaluate_topics(model, feature_names, male_words, female_words):
    topic_male_scores = []
    topic_female_scores = []
    
    for topic_idx, topic_distribution in enumerate(model.components_):
        word_probs = {feature_names[i]: topic_distribution[i] for i in range(len(feature_names))}
        
        # Compute total probability mass assigned to male and female words
        male_score = sum(word_probs.get(word, 0) for word in male_words)
        female_score = sum(word_probs.get(word, 0) for word in female_words)
        
        topic_male_scores.append(male_score)
        topic_female_scores.append(female_score)

    # Find the topic with highest male score and highest female score
    best_male_topic = np.argmax(topic_male_scores)
    best_female_topic = np.argmax(topic_female_scores)

    # If male and female words are in separate topics, return 1, else 0
    return 1 if best_male_topic != best_female_topic else 0

In [18]:
# Fit LDA Model
n_topics = 50  # Adjust based on optimization
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_topics = lda_model.fit_transform(X)

In [81]:
# Show top words per topic
def display_topics(model, feature_names, num_words=50):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]))
        print()

feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names)

Topic 0:
water bottle gold tonic macaroni aqua cucumber light essence refreshingly hill nitro start anchor goose burgess release home morocco spring excel mill guinea dining anticolic flying vale ginger raspberry park rabbit ale blank lemon fill trainer pig tiny pink peter sparkling chard bench black libra tippee premium discovery grey baby

Topic 1:
body wash skin gel lotion face sensitive shower expert men icing cream machine hand drained ring mist naked hair care dry junior health asparagus pistachio hardback shaped dove urban roll oil loreal cleanse ready baby glucosamine mid deep tate day super spray koko wonderland creamery gently elegant cleansing parker male

Topic 2:
ham rich pet tequila deluxe litter plush lotus figure brewery disinfectant cat smooth dog quinoa smoked crate especial nose bed toy famous tray vocation blocked panda cushion furniture spray one chile timer large maestro concentrated hygiene zing shopping aurora jute teat wood blind linen medium fresh tote scoop s

In [1]:
# Loading the package
import pyLDAvis
from pyLDAvis import lda_model
# Enable displaying our model in our notebook
pyLDAvis.enable_notebook()

In [19]:
pyLDAvis.lda_model.prepare(
    lda_model = lda_model,
    dtm = X,
    vectorizer = vectorizer,
    # Keep our original sorting of documents
    sort_topics=False
)

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=74234) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=74234) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=74234) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=74234) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.106968  0.118632       1        1  1.826663
1      0.018797 -0.141035       2        1  1.805860
2     -0.098254  0.005567       3        1  1.792708
3      0.013082  0.048591       4        1  1.745110
4      0.062530  0.046124       5        1  1.733344
5      0.170274  0.048966       6        1  2.468153
6      0.191506  0.072834       7        1  2.479598
7     -0.021164 -0.136229       8        1  1.849607
8      0.160086 -0.068084       9        1  2.391071
9      0.120317  0.122619      10        1  2.128464
10    -0.046699 -0.112750      11        1  1.996738
11    -0.191722  0.109055      12        1  2.316906
12    -0.033099 -0.106387      13        1  1.841861
13     0.031861 -0.058563      14        1  1.975768
14     0.013934 -0.106981      15        1  1.969682
15    -0.049771  0.093639      16        1  2.677322
16    -0.021627 -0.105315      17        1  1.593980
17     0.046222 -0.047645      18        1  1.789599
18     0.061606 -0.035878      19        1  1.839855
19     0.023630  0.016856      20        1  1.663242
20     0.028844  0.022246      21        1  1.896296
21    -0.090980 -0.080427      22        1  2.132093
22     0.205713  0.091301      23        1  2.268820
23     0.088074 -0.057563      24        1  1.784834
24    -0.070222  0.073658      25        1  2.004212
25    -0.004752  0.070580      26        1  2.067314
26     0.177980  0.015888      27        1  1.996080
27     0.220109  0.020766      28        1  2.133385
28    -0.055644  0.210126      29        1  2.271929
29     0.014488 -0.143963      30        1  1.709312
30     0.022332  0.027474      31        1  2.017131
31    -0.036173  0.025769      32        1  1.838087
32    -0.131679  0.031871      33        1  2.170641
33    -0.083452  0.014325      34        1  1.653748
34    -0.051287  0.032266      35        1  1.531707
35    -0.067723  0.068952      36        1  2.329830
36    -0.073594 -0.108107      37        1  2.745996
37     0.014446 -0.084076      38        1  1.819990
38     0.027194  0.019217      39        1  1.677023
39     0.000816  0.061887      40        1  1.708646
40     0.018670  0.209196      41        1  2.603581
41    -0.056151 -0.010251      42        1  1.851151
42     0.062631 -0.084407      43        1  1.780118
43     0.005527 -0.038599      44        1  1.915382
44    -0.142211 -0.083369      45        1  1.800747
45    -0.095974 -0.073193      46        1  1.525389
46    -0.083979 -0.110896      47        1  2.173721
47    -0.209152  0.117281      48        1  2.925574
48     0.030507 -0.063587      49        1  1.569917
49    -0.008895  0.061618      50        1  2.211813, topic_info=          Term        Freq        Total Category  logprob  loglift
5844     sauce  983.000000   983.000000  Default  30.0000  30.0000
7079    tomato  651.000000   651.000000  Default  29.0000  29.0000
1361    coffee  578.000000   578.000000  Default  28.0000  28.0000
5645      rice  704.000000   704.000000  Default  27.0000  27.0000
6882     taste  543.000000   543.000000  Default  26.0000  26.0000
...        ...         ...          ...      ...      ...      ...
2073       dry   59.801975   448.000237  Topic50  -4.4107   1.7976
7585     water   59.988444   529.090804  Topic50  -4.4076   1.6343
2339     extra   49.117625   627.867639  Topic50  -4.6075   1.2632
4697  original   46.497173  1094.686834  Topic50  -4.6624   0.6525
1018   caramel   40.324001   460.740813  Topic50  -4.8048   1.3755

[2591 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0        15  0.974884         aa
3        35  0.900041      abbot
6        18  0.794286   absolute
21       28  0.923649  accessory
37        2  0.050629     action
...     ...       ...        ...
7835     46  0.943418       zest
7836     45  0.975756      zesty
7839     10  0.966927       z